In [157]:
import tensorflow as tf
import tensorflowjs as tfjs
import pandas as pd
import numpy as np

In [150]:
dataTrain = pd.read_csv('./train.csv')
dataTest = pd.read_csv('./test.csv')
dataValidate = pd.read_csv('./validate.csv')
dataTrain.head()

,text,label
0,i won the cricket match,1
1,i ranked first in my class,1
2,i topped in my semesters!,1
3,i didnt feel humiliated,0
4,i am not so happy today,0


In [151]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.oov_token = '<shobdobhandarErBahire>'
tokenizer.fit_on_texts(dataTrain['text'])
vocab = tokenizer.word_index
xTrain = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(dataTrain['text']))
xTest = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(dataTest['text']))
xValidate = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(dataValidate['text']))
yTrain = tf.keras.utils.to_categorical(dataTrain['label'])
yTest = tf.keras.utils.to_categorical(dataTest['label'])
yValidate = tf.keras.utils.to_categorical(dataValidate['label'])
vocabSize = len(vocab)
xTrain.shape, yTrain.shape, vocabSize

((16011, 66), (16011, 6), 15220)

In [152]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocabSize, 64, input_length=66))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy' , optimizer='adam', metrics=['accuracy'])

In [153]:
model.fit(xTrain, yTrain, epochs=15, validation_data=(xValidate, yValidate))

Epoch 1/15
501/501 [==============================] - 15s 20ms/step - loss: 1.1322 - accuracy: 0.5715 - val_loss: 0.6054 - val_accuracy: 0.7870
Epoch 2/15
501/501 [==============================] - 9s 18ms/step - loss: 0.3599 - accuracy: 0.8765 - val_loss: 0.3068 - val_accuracy: 0.8985
Epoch 3/15
501/501 [==============================] - 9s 18ms/step - loss: 0.1603 - accuracy: 0.9455 - val_loss: 0.2619 - val_accuracy: 0.9160
Epoch 4/15
501/501 [==============================] - 10s 21ms/step - loss: 0.0921 - accuracy: 0.9679 - val_loss: 0.2614 - val_accuracy: 0.9210
Epoch 5/15
501/501 [==============================] - 9s 18ms/step - loss: 0.0649 - accuracy: 0.9777 - val_loss: 0.2978 - val_accuracy: 0.9125
Epoch 6/15
501/501 [==============================] - 9s 18ms/step - loss: 0.0533 - accuracy: 0.9816 - val_loss: 0.2997 - val_accuracy: 0.9155
Epoch 7/15
501/501 [==============================] - 9s 18ms/step - loss: 0.0498 - accuracy: 0.9837 - val_loss: 0.3011 - val_accuracy: 0.91

In [154]:
model.evaluate(xTest, yTest)

63/63 [==============================] - 1s 14ms/step - loss: 0.3701 - accuracy: 0.9115


[0.3700506091117859, 0.9114999771118164]

In [155]:
inpt = input('enter the text')
labels_dict = {0:'sadness', 1:'joy', 2:'love', 3:'anger', 4:'fear', 5:'surprise'}
text = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inpt]), padding='pre', maxlen=66)

print(inpt, model.predict(text), labels_dict[np.argmax(model.predict(text))])

mom recovered from covid [[0.01767339 0.03039624 0.00092644 0.15030152 0.79787827 0.00282411]] fear


In [156]:
model.save('ai-model.h5')
tfjs.converters.save_keras_model(model, 'C:\\Users\\007ra\\Documents\\Codes\\AI-ML\\sentiment-analyzer\\AI')